<a href="https://colab.research.google.com/github/xinnazim/esaa/blob/main/%EC%A4%80%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# os 관리를 위한 라이브러리
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

# 데이터 불러오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
train = pd.read_parquet('/content/drive/MyDrive/air/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/air/test_pre.parquet')
sample_submission = pd.read_csv('/content/drive/MyDrive/air/sample_submission.csv', index_col = 0)

In [53]:
train['Delay'].info()

<class 'pandas.core.series.Series'>
Int64Index: 675015 entries, 1 to 999999
Series name: Delay
Non-Null Count   Dtype  
--------------   -----  
172245 non-null  float64
dtypes: float64(1)
memory usage: 10.3 MB


# 준지도학습

## **Self-Training**
* **SelfTrainingClassifier**는 사이킷런의 **BaseEstimator**와 **ClassifierMixin** 클래스를 확장한 사용자 지정 분류기
* 레이블이 없는 데이터를 사용하여 모델을 반복 학습, 가상 레이블 생성
* 사이킷런의 다른 분류기를 기반으로 만들 수 있음
* 일반적으로 기본 분류기는 **레이블이 있는 데이터**로 학습 후 가상 레이블이 있는 데이터로 다시 학습
* 반복하여 모델 업데이트 및 성능 향상


#### chatGPT
```
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.linear_model import LogisticRegression

# 데이터 전처리
X_train = train.drop('delayed', axis=1)
y_train = train['delayed']
X_test = test

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 기본 분류기 생성 (여기서는 로지스틱 회귀를 사용하겠습니다)
base_classifier = LogisticRegression()

# SelfTrainingClassifier 생성
self_training_model = SelfTrainingClassifier(base_classifier, criterion='k_best', k_best=100)

# 모델 학습 (가상 레이블을 생성하고 모델을 업데이트하는 과정이 자동으로 수행됩니다)
self_training_model.fit(X_train_scaled, y_train)

# 예측
y_pred = self_training_model.predict(X_test_scaled)
```



### Step 0.데이터 분리

In [62]:
# 데이터 분리

# 1. 레이블이 있는 데이터와 레이블이 없는 데이터로 분리
X_train_labeled = train.drop('Delay', axis=1)
y_train_labeled = train['Delay']
X_train_unlabeled = test.copy()  # 레이블이 없는 데이터로 테스트 데이터를 사용

# 2. X, y 분리 (레이블이 있는 데이터)
X_train_labeled, X_val_labeled, y_train_labeled, y_val_labeled = train_test_split(X_train_labeled, y_train_labeled, test_size=0.2, random_state=42)


In [64]:
train = pd.read_parquet('/content/drive/MyDrive/air/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/air/test_pre.parquet')
train_labeled = train.loc[train['Delay'].notnull(),:]
train_unlabeled = train.loc[train['Delay'].isnull(),:]

In [66]:
# train
X_labeled = train_labeled.drop(['Delay'],axis = 1)
y_labeled = train_labeled['Delay']
X_unlabeled = train_unlabeled.drop(['Delay'],axis = 1)
X_test= test

In [70]:
# 5. Self-training 수행

base_classifier = RandomForestClassifier()
self_training_model = SelfTrainingClassifier(base_classifier, criterion='k_best', k_best=10)

# Step 1. Base Model 학습
self_training_model.fit(X_labeled, y_labeled)

# Step 2. Self-training을 통해 의사 레이블 생성
pseudo_labels = self_training_model.predict(X_unlabeled)

/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:217: UserWarning: k_best is larger than the amount of unlabeled samples. All unlabeled samples will be labeled in the first iteration
  warnings.warn(


In [71]:
pseudo_labels # 왜??

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
# Step 3. 의사 레이블을 기존 레이블이 있는 데이터에 추가하여 새로운 학습용 데이터 구성
X_labeled_augmented = pd.concat([X_labeled, X_unlabeled])
y_labeled_augmented = pd.concat([y_labeled, pd.Series(pseudo_labels)])

# Step 4. 모델 재학습
self_training_model.fit(X_labeled_augmented, y_labeled_augmented)

# Step 5. 테스트 데이터에 대한 예측 수행
y_pred = self_training_model.predict(X_test)

# 결과 출력
print(y_pred)

* 의사 레이블(Pseudo label)은 준지도학습에서 사용되는 개념으로, 레이블이 없는 데이터에 대해 모델이 예측한 가상의 레이블

In [41]:
# 6. 예측
# Self-training을 완료한 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred = self_training_model.predict(X_val_labeled_scaled)

# 결과 출력
print(y_pred)

[0. 0. 0. ... 0. 0. 0.]


잘못함

* 준지도학습(Self-training)을 활용하여 NaN 값을 의사 레이블로 취급하고 모델링


In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier

# Step 0. 데이터 분리
# 데이터 불러오기
train = pd.read_parquet('/content/drive/MyDrive/air/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/air/test_pre.parquet')

#새로운 데이터프레임 train_st와 test_st를 생성
train_st = train.copy()
test_st = test.copy()

# 레이블 인코딩 수행
label_encoder = LabelEncoder()
train_st['Delay'] = label_encoder.fit_transform(train_st['Delay'])

# 학습 데이터와 레이블 분리
X_train_st = train_st.drop('Delay', axis=1)
y_train_st = train_st['Delay']

# SelfTrainingClassifier 생성
base_classifier = RandomForestClassifier()  # 기본 분류기로 RandomForestClassifier를 사용
self_training_model = SelfTrainingClassifier(base_classifier, criterion='k_best', k_best=30)

# Self-training 수행
self_training_model.fit(X_train_st, y_train_st)

/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:217: UserWarning: k_best is larger than the amount of unlabeled samples. All unlabeled samples will be labeled in the first iteration
  warnings.warn(


SelfTrainingClassifier(base_estimator=RandomForestClassifier(),
                       criterion='k_best', k_best=30)

In [60]:
# 레이블이 없는 데이터에 대한 예측 수행
X_test_st=test_st
pseudo_labels = self_training_model.predict(X_test_st)

# 예측 결과 출력
print(pseudo_labels)

[1 0 2 ... 2 2 2]


In [61]:
pseudo_labels

array([1, 0, 2, ..., 2, 2, 2])

* k_best 값은 Self-training 과정에서 의사 레이블로 변환할 때 사용되는 기준
* 즉, Self-training의 각 반복에서 가장 확신 있는 상위 k_best개의 샘플을 의사 레이블로 선택하게 됩니다. 이 k_best 값을 어떻게 설정하느냐에 따라 Self-training의 성능과 학습 속도에 영향

* 보통은 작은 값 (예: 10, 20, 50)으로 시작하여 여러 가지 값을 시도





